Import the libraries required

In [59]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from bs4 import BeautifulSoup as BS
import folium
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Using Beautiful Soup to Scrap the website and reading information in table to variable t. Information about Postal_code, Borough and Neighbourhoold is recursively stored in respective list

In [2]:
#toronto=pd.read_csv('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').json()
toronto_site = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
toronto_data=BS(toronto_site,'html.parser')
t=toronto_data.find('table').find_all('tr')
Postal_Code=[]
Borough=[]
Neighborhood=[]
#toronto_data
for i in t:
    #j.append(i.text)
    Postal_Code.append(i.text.splitlines()[1])
    Borough.append(i.text.splitlines()[3])
    Neighborhood.append(i.text.splitlines()[5])

Converting List in Pandas Dataframe

In [3]:
Toronto_Dataframe=pd.DataFrame({'PostalCode':Postal_Code,'borough':Borough,'neighborhood':Neighborhood})

Removing Non assigned borough

In [4]:

Clean_Dataframe=Toronto_Dataframe[Toronto_Dataframe['borough']!='Not assigned'].reset_index(drop=True)
Clean_Dataframe.head()

,PostalCode,borough,neighborhood
0,Postal Code,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


Joining neighbourhood with same Postalcode

In [5]:
d=Clean_Dataframe.groupby(['PostalCode','borough'])['neighborhood'].apply(lambda x: ','.join(x)).reset_index()

Assigning Not assigned neighbourhood with name of borough

In [6]:
for ind in d.index:
    if d['neighborhood'][ind]=='Not assigned':
        d['neighborhood'][ind]=d['borough'][ind]
d.head(-5)

,PostalCode,borough,neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
94,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov..."
95,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Humberlea, Emery"


In [7]:
   
    d.shape     

(104, 3)

In [8]:
file1=pd.read_csv('Geospatial_Coordinates.csv')
file1

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
d['Latitude']=file1['Latitude']
d['Longitude']=file1['Longitude']

In [10]:
d.head()

,PostalCode,borough,neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
address='Toronto,Ca'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

The geograpical coordinate of toronto are 43.6534817, -79.3839347.


In [14]:
j=d[d['borough']=='Scarborough' ].reset_index(drop=True)
j
for lat, lng, borough, neighborhood in zip(j['Latitude'], j['Longitude'], j['borough'], j['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto
# explore Scarborough

In [15]:
CLIENT_ID = 'LODUGKCISBXROUZ4JBMBRK2T5QBAE24UASAJ4D1JU5BJP1MM' # your Foursquare ID
CLIENT_SECRET = 'O5TWVAKSAFU0R4Q1O04NTMJAR05JDO2Z4IWT4BS3HHQWFW01' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LODUGKCISBXROUZ4JBMBRK2T5QBAE24UASAJ4D1JU5BJP1MM
CLIENT_SECRET:O5TWVAKSAFU0R4Q1O04NTMJAR05JDO2Z4IWT4BS3HHQWFW01


Lets explore first neighbourhood

In [16]:
j.loc[1, 'neighborhood']

'Rouge Hill, Port Union, Highland Creek'

Lets explore 20 venues around Malvern Rouge

In [23]:
neighborhood_latitude = j.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = j.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = j.loc[1, 'neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))
LIMIT=50
radius=500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

Latitude and longitude values of Rouge Hill, Port Union, Highland Creek are 43.7845351, -79.16049709999999.


In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Royal Canadian Legion,Bar,43.782533,-79.163085
1,Affordable Toronto Movers,Moving Target,43.787919,-79.162977


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
Scarborough_venues = getNearbyVenues(names=j['neighborhood'],
                                   latitudes=j['Latitude'],
                                   longitudes=j['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [27]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(91, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Number of time a venue is repeated

In [28]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",7,7,7,7,7,7
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


Lets analyze the venue

In [30]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 57 uniques categories.


Lets analyze each neighbourhood

In [31]:
scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
scarborough_onehot.shape

(91, 58)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.20,0.00,0.0,0.000000,0.000000,0.0,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.25,...,0.000000,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.0,0.000000
2,Cedarbrae,0.0,0.111111,0.111111,0.111111,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.111111,0.000000,0.0,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000000,0.000000,0.071429,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.071429,0.00,0.00,0.0,0.071429,0.000000,0.0,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.1,0.000000,0.000000,0.0,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.000000,0.000000,0.142857,0.0,0.142857,0.0,0.0,0.00,...,0.142857,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.000000
8,"Kennedy Park, Ionview, East Birchmount Park",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.2,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.2,0.000000
9,"Malvern, Rouge",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.000000


Let's print each neighborhood along with the top 3 most common venues

In [34]:
num_top_venues = 3

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25


----Cedarbrae----
                 venue  freq
0               Lounge  0.11
1      Thai Restaurant  0.11
2  Fried Chicken Joint  0.11


----Clarks Corners, Tam O'Shanter, Sullivan----
           venue  freq
0    Pizza Place  0.14
1  Shopping Mall  0.07
2       Pharmacy  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1                Motel   0.5
2   Italian Restaurant   0.0


----Dorset Park, Wexford Heights, Scarborough Town Centre----
                   venue  freq
0      Indian Restaurant  0.29
1  Vietnamese Restaurant  0.14
2     Chinese Restaurant  0.14


----Golden Mile, Clairlea, Oakridge---

Putting it into data frame

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Gas Station
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Construction & Landscaping,Hakka Restaurant,Grocery Store,Gas Station,Furniture / Home Store
2,Cedarbrae,Hakka Restaurant,Bank,Fried Chicken Joint,Caribbean Restaurant,Lounge,Gas Station,Bakery,Thai Restaurant,Athletics & Sports,Grocery Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Noodle House,Shopping Mall,Chinese Restaurant,Convenience Store,Pharmacy,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Bank
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Construction & Landscaping,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


Running k-mean cluster to cluster them into 5 groups

In [41]:
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 3, 0, 0, 0, 2, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [50]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Gas Station
1,0,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Construction & Landscaping,Hakka Restaurant,Grocery Store,Gas Station,Furniture / Home Store
2,0,Cedarbrae,Hakka Restaurant,Bank,Fried Chicken Joint,Caribbean Restaurant,Lounge,Gas Station,Bakery,Thai Restaurant,Athletics & Sports,Grocery Store
3,0,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Noodle House,Shopping Mall,Chinese Restaurant,Convenience Store,Pharmacy,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Bank
4,3,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Construction & Landscaping,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
5,0,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Chinese Restaurant,Light Rail Station,Furniture / Home Store,Pet Store,Vietnamese Restaurant,Breakfast Spot,Bus Line,Hakka Restaurant,Grocery Store
6,0,"Golden Mile, Clairlea, Oakridge",Bus Line,Bakery,Intersection,Ice Cream Shop,Park,Metro Station,Bus Station,Soccer Field,Furniture / Home Store,Gas Station
7,0,"Guildwood, Morningside, West Hill",Intersection,Bank,Breakfast Spot,Rental Car Location,Medical Center,Electronics Store,Mexican Restaurant,Convenience Store,Grocery Store,General Entertainment
8,2,"Kennedy Park, Ionview, East Birchmount Park",Train Station,Bus Station,Discount Store,Coffee Shop,Department Store,Convenience Store,Ice Cream Shop,Hakka Restaurant,Grocery Store,General Entertainment
9,1,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store


In [53]:
scarborough_merged = j
scarborough_merged

,PostalCode,borough,neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [56]:
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhood')


ValueError: columns overlap but no suffix specified: Index(['Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

In [75]:
scarborough_merged_clean=scarborough_merged[scarborough_merged['neighborhood']!='Upper Rouge']

In [62]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



In [82]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster1 in zip(scarborough_merged_clean['Latitude'], scarborough_merged_clean['Longitude'], scarborough_merged_clean['neighborhood'], scarborough_merged_clean['Cluster Labels']):
    cluster=int(cluster1)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

1
4
0
2
0
2
2
0
3
0
0
0
0
0
2
0
